In [1]:
from kubeflow import fairing
from kubeflow.fairing.kubernetes import utils as k8s_utils
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource
import glob

In [6]:
fairing.config.set_preprocessor(
    'python', 
    path_prefix="/", # 이 디렉토리로 복사해서 압축한다
#     command=["python", "/train/mnist.py"],  # 생략하면 Job Pod 실행 안함 뭥미 Dockerfile의 CMD는 그냥 무시하남
    input_files = glob.glob("mnist/**", recursive=True),
)

context_source = MinioContextSource(
    endpoint_url='http://minio-service.kubeflow:9000',
    minio_secret='minio',
    minio_secret_key='minio123',
    region_name='fairing' # f"/kubeflow-{region_name}"으로 버킷 생성
)

fairing.config.set_builder(
    'cluster',
    registry = 'registry.kube-system.svc.cluster.local:30000',
    image_name = "my-first-cluster-build", 
    context_source = context_source,
    dockerfile_path="mnist/Dockerfile",
    push=True,
    # cleanup=True
)

# kaniko_image = 'gcr.io/kaniko-project/executor:v0.22.0' # 원래 값
kaniko_image = 'gcr.io/kaniko-project/executor:latest'
fairing.constants.constants.KANIKO_IMAGE = kaniko_image
    
fairing.config.set_deployer(
    'job',
    # job_name='my-python-file-fairing',
    # namespace='myspace', # default: 현재 네임스페이스
    # pod_spec_mutators=[
    #     k8s_utils.get_resource_mutator(cpu=1, memory=5)
    # ]
)

fairing.config.run()

[I 210719 15:44:33 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.base.BasePreProcessor object at 0x7f205669a160>
[I 210719 15:44:33 config:136] Using builder: <kubeflow.fairing.builders.cluster.cluster.ClusterBuilder object at 0x7f20566b6668>
[I 210719 15:44:33 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f205669a1d0>
[I 210719 15:44:33 cluster:46] Building image using cluster builder.
[I 210719 15:44:33 base:107] Creating docker context: /tmp/fairing_context_79lai687


S3UploadFailedError: Failed to upload /tmp/fairing_context_79lai687 to kubeflow-fairing/fairing-builds/BC6E69FC: An error occurred (XMinioStorageFull) when calling the PutObject operation: Storage backend has reached its minimum free disk threshold. Please delete a few objects to proceed.

# curl로 프라이빗 레지스트리 확인

In [3]:
! curl http://registry.kube-system.svc.cluster.local:30000/v2/_catalog

{"repositories":["my-02-python-file-fairing","my-05-build-only-fairing","my-first-cluster-build"]}


In [4]:
! curl http://registry.kube-system.svc.cluster.local:30000/v2/my-first-cluster-build/tags/list

{"name":"my-first-cluster-build","tags":["D8F189F7","D46A1DCD","FE8B3613","57A335DD","A3DA3AF4","B635F8A1","C8FE01FA","E61D212","CCB57B7A","F2888ED4","7EDC9B7A","A21D8044","4B031CE7","FC62AB31","CE50DBA6","4058D583","CC7B1E27","7E25F8A9","B57FE51","F6EE5264","DD4D94C8","13541630","A77C105F","EB92B64F","905AFC42","DA82D7E5"]}


# Image 확인
- Host에서 실행
- fairing에서 난수로 생성한 tag를 변경해서 실행할 것 (예: E2683407)

### Image pull
```sh
docker pull registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```

### Image inspect
```sh
docker inspect registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```
Cmd 항목을 확인하고 faring 코드대로 image build 되었음을 확인
```json
...
            "Cmd": [
                "python",
                "/app/-python-file-to-fairing.py"
            ],
...
```